# Initial scrip for Titanic Kaggle competition

## Data Loading and initial overview

In [117]:
import numpy as np  
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns   
import torch 
import torch.nn as nn
import torch.nn.functional as F
import os

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

tqdm.pandas()

%matplotlib inline

In [53]:
dire = 'titanic_data/'
train_df = pd.read_csv(f'{dire}train.csv')   
test_df = pd.read_csv(f'{dire}test.csv')   

In [54]:
train_df.head() 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [56]:
train_df.describe() 

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## One-hot encoding

In [61]:
train_df_oh = pd.get_dummies(train_df,  columns=['Sex', 'Embarked'], dtype=int)

## Cabin encodin

In [62]:
def encode_cabin(df: pd.DataFrame,
                 ) -> pd.DataFrame:
    
    def supp_encode_cab(element):
        if element is not np.nan:
            letter = element[0]
            rest = element[1:]
            return letter, rest
        return np.nan, np.nan
    
    df[['letter', 'rest']] = df['Cabin'].apply(supp_encode_cab).apply(pd.Series)
    df.drop(columns=['Cabin'], inplace=True)
    
    return df

In [63]:
train_df_oh = encode_cabin(train_df_oh)

In [64]:
train_df_oh

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,letter,rest
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,0,1,0,0,1,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,1,0,1,0,0,C,85
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,1,0,0,0,1,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,1,0,0,0,1,C,123
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,0,1,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,0,1,0,0,1,NaN,NaN
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,1,0,0,0,1,B,42
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,1,0,0,0,1,NaN,NaN
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,0,1,1,0,0,C,148


In [65]:
train_df_oh = pd.get_dummies(train_df_oh, columns=['letter'], dtype=int)

## Touchdown

In [ ]:
train_df_oh.drop(columns=['Name', 'Ticket', 'PassengerId'], inplace=True)

In [82]:
def type_change(element):
    try:
        element = int(element)
    except:
        element = np.nan
    return element

In [84]:
train_df_oh['rest'] = train_df_oh['rest'].apply(type_change)

In [89]:
train_df_oh = train_df_oh.astype(float)

## Torching

In [99]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available device: {device}')

Available device: cuda


In [100]:
data = train_df_oh.to_numpy()
data = torch.tensor(data, dtype=torch.float32, device=device)

In [103]:
samples, features = data.shape
val_size = 0.2
v_split = int(val_size * samples)
t_split = samples - v_split
idx = torch.randperm(samples)
data = data[idx, :]
train_data, val_data = torch.split(data, [t_split, v_split])

In [148]:
from torch.utils.data import Dataset, DataLoader

class TitanicDataset(Dataset):
    
    def __init__(self, 
                 data: torch.tensor):
        super(TitanicDataset).__init__()
        self.data = data
        self.samples, self.features = data.shape
        
    def __len__(self):
        return self.samples
    
    def __getitem__(self, idx):
        y = self.data[idx, 0]
        X = self.data[idx, 1:]
        return X, y

In [149]:
train_dataset = TitanicDataset(train_data)
val_dataset = TitanicDataset(val_data)

print(len(train_dataset))
print(len(val_dataset))

713
178


In [150]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [151]:
X, y = next(iter(train_loader))
X.shape

torch.Size([16, 19])

In [152]:
## Model

In [153]:
class LinearNN(nn.Module):
    
    '''Regular linear neural net model'''
    
    def __init__(self, 
                 layer_sizes: int,):
        super().__init__()
        self.layer_sizes = layer_sizes
        layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            layer = nn.Linear(layer_sizes[i], layer_sizes[i+1], bias=True)
            layers.append(layer)
        self.layers = layers
    
    def forward(self,
                x: torch.tensor) -> torch.tensor:
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers) - 1:
                x = F.relu(x)
            else:
                x = F.sigmoid(x) # Binary classification
        return x

## Training

In [ ]:
def train(model: nn.Module,
          optimizer: torch.optim.AdamW,
          loss_fn: F,
          train_loader: DataLoader,
          val_loader: DataLoader,
          epochs: int=100,
          device:torch.device=device,) -> tuple:
    
    train_losses = []
    val_losses = []
    
    writer = SummaryWriter(log_dir='runs/training_logs')
    model = model.to(device)
    
    for epoch in tqdm(range(epochs)):
        avg_loss = 0
        model.train()
        
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            
        avg_loss /= len(train_loader)
        train_losses.append(avg_loss)
        
        writer.add_scalar('Loss/Train', avg_loss, epoch)
        
        avg_val_loss = 0
        
        with torch.no_grad():
            model.eval()
            for X, y in val_loader:
                pred = model(X)
                loss = loss_fn(pred, y)
                avg_val_loss += loss.item()
                
            avg_val_loss /= len(val_loader)
            val_losses.append(avg_val_loss)
        
        writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
        
    writer.close()
    
    return train_losses, val_losses

In [159]:
model = LinearNN(layer_sizes=[19, 50, 100, 50, 1])
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
loss_fn = nn.MSELoss()

In [160]:
train_losses, val_losses = train(model, 
                                 optimizer,
                                 loss_fn,
                                 train_loader,
                                 val_loader,
                                 epochs=10,
                                 device=device)

  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\filip\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\filip\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\filip\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they hav

In [162]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.grid()
plt.legend()
plt.title('Loss')
plt.xlabel('Epoch')

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.